# Initialize the accelerator

In [ ]:
from finn_examples import models
print(list(filter(lambda x: "mnist" in x, dir(models))))

In [ ]:
accel = models.tfc_w1a1_mnist()

In [ ]:
print("Expected input shape and datatype: %s %s" % (str(accel.ishape_normal()), str(accel.idt())))
print("Expected output shape and datatype: %s %s" % (str(accel.oshape_normal()), str(accel.odt())))

# Load the MNIST dataset

Use the `dataset_loading` package to get easy Python access to MNIST dataset:

In [ ]:
from dataset_loading import mnist
trainx, trainy, testx, testy, valx, valy = mnist.load_mnist_data("/tmp", download=True, one_hot=False)

In [ ]:
testx.shape

# Classify a single image

In [ ]:
test_single_x = testx[0].reshape(28, 28)
test_single_y = testy[0]

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.imshow(test_single_x, cmap='gray')
plt.show()

In [ ]:
print("Expected class is %d" % test_single_y)

In [ ]:
accel_in = test_single_x.reshape(accel.ishape_normal())
print("Input buffer shape is %s and datatype is %s" % (str(accel_in.shape), str(accel_in.dtype)))

In [ ]:
accel_out = accel.execute(accel_in)

In [ ]:
print("Returned class is %d" % accel_out)

In [ ]:
%%timeit
accel_out = accel.execute(accel_in)

# Validate accuracy on entire MNIST test set

In [ ]:
import numpy as np
batch_size = 1000
total = testx.shape[0]
accel.batch_size = batch_size
n_batches = int(total / batch_size)

batch_imgs = testx.reshape(n_batches, batch_size, -1)
batch_labels = testy.reshape(n_batches, batch_size)
obuf_normal = np.empty_like(accel.obuf_packed_device)
print("Ready to run validation, test images tensor has shape %s" % str(batch_imgs.shape))
print("Accelerator buffer shapes are %s for input, %s for output" % (str(accel.ishape_packed()), str(accel.oshape_packed())) )

In [ ]:
ok = 0
nok = 0
for i in range(n_batches):
    ibuf_normal = batch_imgs[i].reshape(accel.ishape_normal())
    exp = batch_labels[i]
    obuf_normal = accel.execute(ibuf_normal)
    ret = np.bincount(obuf_normal.flatten() == exp.flatten())
    nok += ret[0]
    ok += ret[1]
    print("batch %d / %d : total OK %d NOK %d" % (i, n_batches, ok, nok))

In [ ]:
acc = 100.0 * ok / (total)
print("Final accuracy: {}%".format(acc))

In [ ]:
def run_validation():
    for i in range(n_batches):
        ibuf_normal = batch_imgs[i].reshape(accel.ishape_normal())
        exp = batch_labels[i]
        accel.execute(ibuf_normal)

In [ ]:
full_validation_time = %timeit -n 5 -o run_validation()

In [ ]:
print("%f images per second including data movement" % (total / float(full_validation_time.best)))

## Run some more built-in benchmarks

In [ ]:
accel.throughput_test()